In [10]:
import tensorflow as tf

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data() # MNISTデータセットをロードして準備
x_train, x_test = x_train / 255.0, x_test / 255.0 # 整数から浮動小数点数に変換

In [11]:
# 層を積み重ねてモデルを構築
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

In [12]:
predictions = model(x_train[:1]).numpy() # クラスごとにロジットや対数オッズ比と呼ばれるスコアを算出
predictions

array([[-0.1199351 , -0.24876107, -0.55953145, -0.12498512,  0.5667242 ,
         0.10201988, -0.04447686,  0.09132677,  0.15825573, -0.08522455]],
      dtype=float32)

In [13]:
tf.nn.softmax(predictions).numpy() # クラスごとにこれらのロジットを確立に変換

array([[0.08753778, 0.07695682, 0.05640028, 0.08709683, 0.17394337,
        0.10929225, 0.09439884, 0.10812981, 0.1156145 , 0.09062961]],
      dtype=float32)

In [15]:
# ロジットとTrueのインデックスに関するベクトルを入力にとり，それぞれの標本についてクラスごとに損失のスカラーを返す
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [18]:
# この損失はクラスが正しい確率の大数をとって符号を判定させたものである，この値はモデルがこのクラスが正しいと確信しているときに0になる。
# この訓練されていないモデルはランダムに近い値(それぞれのクラスについて1/10)を出力する。
loss_fn(y_train[:1], predictions).numpy()

2.2137299

In [19]:
model.compile(optimizer = 'adam',
             loss=loss_fn,
             metrics=['accuracy'])

In [21]:
# 損失を最小化するようにモデルのパラメータを調整
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 1s 739us/step - loss: 0.0642 - accuracy: 0.9793
Epoch 2/5
1875/1875 [==============================] - 1s 726us/step - loss: 0.0572 - accuracy: 0.9812
Epoch 3/5
1875/1875 [==============================] - 1s 721us/step - loss: 0.0517 - accuracy: 0.9830
Epoch 4/5
1875/1875 [==============================] - 1s 722us/step - loss: 0.0456 - accuracy: 0.9851
Epoch 5/5
1875/1875 [==============================] - 1s 736us/step - loss: 0.0448 - accuracy: 0.9847


In [23]:
# モデルの性能を検査する。
# これは通常検証用データセットまたはテストデータセットを用いる
model.evaluate(x_test, y_test, verbose=2)

313/313 - 0s - loss: 0.0692 - accuracy: 0.9796 - 146ms/epoch - 467us/step


[0.06920691579580307, 0.9796000123023987]

In [24]:
# モデルが確率を返すようにする場合
probability_model = tf.keras.Sequential([
    model,
    tf.keras.layers.Softmax()
])
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[9.13719309e-11, 8.09100963e-12, 2.68370925e-08, 2.24170217e-05,
        2.03961531e-15, 1.56127139e-10, 1.04326255e-19, 9.99977469e-01,
        1.73540363e-10, 9.99832750e-08],
       [5.30501580e-08, 3.57030149e-06, 9.99995708e-01, 7.25625782e-08,
        6.84837951e-20, 4.00669194e-07, 1.14928490e-07, 1.36217859e-15,
        3.31509931e-08, 2.22879091e-16],
       [6.40352482e-09, 9.99957800e-01, 1.19124879e-05, 5.29031219e-09,
        1.13095746e-06, 2.97880351e-08, 2.07652278e-07, 2.28854278e-05,
        5.90343416e-06, 1.65587988e-09],
       [9.99940395e-01, 5.24328525e-10, 2.34442814e-06, 4.64551064e-08,
        3.33389183e-09, 3.17129457e-06, 5.37625710e-05, 1.17912627e-07,
        4.30110288e-11, 1.27491404e-07],
       [1.38667176e-07, 2.73876448e-14, 2.93018161e-08, 2.08166959e-08,
        9.99953270e-01, 9.57238822e-09, 6.88040345e-08, 2.28890690e-06,
        2.00153161e-09, 4.41953998e-05]], dtype=float32)>